In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import cPickle as pk
np.random.seed(1) # to be reproductive

In [2]:
# paths
NOTE_DATA_DIR = '/local/XW/DATA/MIMIC/noteevents_by_sid/'
ICD_FPATH = 'data/subject_diag_icds.txt'
PK_FPATH = 'data/diag_processed_data.pk' # './processed_data_small.pk'
N_LABELS = 50

In [3]:
# read k-hot labels data
pk_data = pk.load(open(PK_FPATH, 'rb'))
Y_train = pk_data['Y_train']
Y_val = pk_data['Y_val']

In [4]:
Y_train.shape

(36917, 50)

In [5]:
pk_data.keys()

['embedding_matrix',
 'X_train',
 'X_val',
 'Y_val',
 'Y_train',
 'sid2khot',
 'sid2seq']

In [7]:
def multilabel_evaluate(y_pred, y_true=Y_val):
    y_pred, y_true = y_pred[:,:-1], y_true[:,:-1] # test without last column considered
#     print y_pred.shape, y_true.shape
    tp = np.sum(y_true * y_pred, axis=-1) 
    sum_true = np.sum(y_true, axis=-1)
    sum_pred = np.sum(y_pred, axis=-1)
    union = np.sum(np.clip(y_true+y_pred, 0, 1), axis=-1)
    print 'precision =', np.mean(tp/(sum_pred+1e-10))
    print 'recall = ', np.mean(tp/(sum_true+1e-10))
    print 'F1 = ', 2*np.mean(tp/(sum_true+sum_pred+1e-10))
    print 'acc = ', np.mean(tp/(union+1e-10))

# Naive baseline

In [14]:
print Y_train.shape, Y_val.shape
# print Y_train[:2,]
stat = Y_train.sum(axis=0)
print stat# naive algo just find which label is most common, then always predict this label
pred_naive = stat > Y_train.shape[0]/2
pred_naive = np.array(pred_naive, dtype=np.float64)
# print pred_naive
pred_naive = np.tile(pred_naive, (Y_val.shape[0],1))
# print pred_naive.shape
multilabel_evaluate(pred_naive)

(36917, 50) (9229, 50)
[ 14021.   8595.   8119.   7855.   6168.   5990.   5872.   5258.   4578.
   4558.   4390.   4220.   4196.   3952.   3483.   3409.   3296.   3015.
   2882.   2810.   2829.   2718.   2670.   2539.   2362.   2304.   2348.
   2286.   2225.   2198.   2226.   2214.   2175.   2128.   2028.   2020.
   1897.   1852.   1854.   1866.   1847.   1815.   1787.   1713.   1642.
   1642.   1638.   1591.   1599.  35142.]
precision = 0.0
recall =  0.0
F1 =  0.0
acc =  0.0


# SVM baseline

** The problem with SVM is that for each instance, there are several labels, so I split each instance into several (instance, one-label) pairs... NOT SURE IF THIS MAKES SENSE. **

This baseline use the method described at: http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [22]:
# # prepare label data
# N_LABELS = 50
# K_ICDS_TOKEEP = N_LABELS - 1 # predict only on top K frequent icd codes
# N_SUBJECTS = 46146
# from collections import Counter
# sid2icds = {} # map subject_id ---> icd codes of this patient
# icd_ctr = Counter()
# with open(ICD_FPATH) as f: 
#     for line in tqdm(f, total=N_SUBJECTS): 
#         sid, _icds = line.split(',')
#         _icds = _icds.split()
#         icd_ctr.update(_icds)
#         sid2icds[sid] = set(_icds)
# print icd_ctr.most_common(K_ICDS_TOKEEP)
# icds = zip( *icd_ctr.most_common(K_ICDS_TOKEEP) )[0] + ('other',)
# sid2khot = {} # map subject_id to k-hot vector
# for sid in sid2icds.keys():
#     _khot = np.zeros(N_LABELS)
#     for _icd in sid2icds[sid]:
#         if _icd in icds: 
#             _khot[icds.index(_icd)] = 1
#         else: # label 'other icds'
#             _khot[-1] = 1
#     sid2khot[sid] = _khot
# sids = sid2icds.keys()
# np.random.shuffle(sids)
# VALIDATION_SPLIT = 0.2
# validset_sz = int(VALIDATION_SPLIT*len(sids))
# train_sids = sids[:-validset_sz] 
# val_sids = sids[-validset_sz:]

# Y_train = np.array([sid2khot[sid] for sid in train_sids])
# Y_val = np.array([sid2khot[sid] for sid in val_sids])
# print Y_train.shape, Y_val.shape

# # To prepare training data for svm (tdidf), need to write our own generator of documents.
# def notes_generator(fpaths):
#     for fpath in tqdm(fpaths):
#         df = pd.read_csv(fpath)
#         yield '\n=======\n\n\n'.join(df['text'])
# from sklearn.feature_extraction.text import CountVectorizer
# MAX_NB_WORDS = 20000 # top 20k most freq words
# cnt_vect = CountVectorizer(max_features=MAX_NB_WORDS)

# train_files = [os.path.join(NOTE_DATA_DIR, '%s.csv'%sid) for sid in train_sids]
# X_train_counts = cnt_vect.fit_transform(notes_generator(train_files))
# print X_train_counts.shape
# from sklearn.feature_extraction.text import TfidfTransformer
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# print X_train_tfidf.shape

# val_files = [os.path.join(NOTE_DATA_DIR, '%s.csv'%sid) for sid in val_sids]
# X_val_counts = cnt_vect.fit_transform(notes_generator(val_files))
# print X_val_counts.shape
# tfidf_transformer = TfidfTransformer()
# X_val_tfidf = tfidf_transformer.fit_transform(X_val_counts)
# print X_val_tfidf.shape

# print X_train_tfidf.shape, Y_train.shape
# Y_train_labels = [Y_train[:,i] for i in xrange(Y_train.shape[1])]

In [23]:
# description = '''
# this file contains the tfidf vectors for svm baseline'''
# data_svm = {
#     'description': description,
#     'X_train': X_train_tfidf,
#     'Y_train': Y_train,
#     'X_val': X_val_tfidf,
#     'Y_val': Y_val,
#     'train_sids': train_sids,
#     'val_sids': val_sids
# }
# pk.dump(data_svm, open('data/data_tfidf_svm.pk','wb'), pk.HIGHEST_PROTOCOL)

In [21]:
data_svm = pk.load(open('data/data_tfidf_svm.pk', 'rb'))
data_svm.keys()

['X_val',
 'X_train',
 'description',
 'Y_val',
 'Y_train',
 'val_sids',
 'train_sids']

In [ ]:
X_train_tfidf = data_svm['X_train']
Y_train = data_svm['Y_train']

**Train 50 independent SVM classifiers**.   
To prepare training data for svm (tdidf), need to write our own **generator** of documents. 

** generate 0/1 label for each class ** 

In [46]:
clfs2 = []
for i in tqdm(range(N_LABELS)):
#     clf = SVC(class_weight='balanced',random_state=1)
    clf =  SGDClassifier(loss='hinge', penalty='l2', 
                   class_weight='balanced',
                   alpha=1e-3, n_iter=200, random_state=1)
    clf.fit(X_train_tfidf, Y_train_labels[i]) 
    clfs2.append(clf)
preds = [clfs2[i].predict(X_val_tfidf) for i in xrange(N_LABELS)]    
pred_svm2 = np.vstack(preds).T
print pred_svm2.shape
multilabel_evaluate(pred_svm2)

100%|██████████| 50/50 [24:09<00:00, 26.33s/it]


(9229, 50)
precision = 0.056609600168
recall =  0.0166419177699
F1 =  0.0233054839194
acc =  0.0158157916279


In [29]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
clfs = []
for i in tqdm(range(N_LABELS)):
#     clf = SVC(class_weight='balanced',random_state=1)
    clf =  SGDClassifier(loss='hinge', penalty='l2', 
                   class_weight='balanced',
                   alpha=1e-2, n_iter=200, random_state=1)
    clf.fit(X_train_tfidf, Y_train_labels[i]) 
    clfs.append(clf)
preds = [clfs[i].predict(X_val_tfidf) for i in xrange(N_LABELS)]    
pred_svm = np.vstack(preds).T
print pred_svm.shape
multilabel_evaluate(pred_svm)

100%|██████████| 50/50 [23:16<00:00, 24.07s/it]


(9229, 50)
precision = 0.101886031631
recall =  0.292810015915
F1 =  0.138220455238
acc =  0.0802666842077


### Best result for svm-tfidf vec:

In [30]:
clfs3 = []
for i in tqdm(range(N_LABELS)):
#     clf = SVC(class_weight='balanced',random_state=1)
    clf =  SGDClassifier(loss='hinge', penalty='l2', 
                   class_weight='balanced',
                   alpha=1e-1, n_iter=200, random_state=1)
    clf.fit(X_train_tfidf, Y_train_labels[i]) 
    clfs3.append(clf)
preds = [clfs3[i].predict(X_val_tfidf) for i in xrange(N_LABELS)]    
pred_svm3 = np.vstack(preds).T 
print pred_svm3.shape
multilabel_evaluate(pred_svm3)

100%|██████████| 50/50 [25:04<00:00, 29.73s/it]


(9229, 50)
precision = 0.112945553919
recall =  0.517047153819
F1 =  0.174670895744
acc =  0.103387535765


# Deep patient baseline

In [31]:
VALIDATION_SPLIT = 0.2

In [32]:
sid2khot = pk_data['sid2khot'] # maps sid(int) to khot encoding of Y_true

In [33]:
sid2khot[2]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [34]:
data = pk.load(open('./data/feature_DP_st_cui.pk'))
dpvecs = data['Xdp']
sid2rowidx = data['sid2rowidx'] # map sid(int) to row index in deep patient feature matrix

In [35]:
len(sid2rowidx), len(sid2khot)

(46146, 46146)

In [36]:
sids = np.array(sid2rowidx.keys())
np.random.shuffle(sids)
print sids

[79423 22600 88696 ...,  1326  2368 27617]


In [37]:
sid_train, sid_val = sids[int(len(sids)*VALIDATION_SPLIT):], sids[:int(len(sids)*VALIDATION_SPLIT)]

In [38]:
len(sid_train), len(sid_val)

(36917, 9229)

In [39]:
X_train, Y_train = [],[]
for sid in sid_train:
    X_train.append(dpvecs[sid2rowidx[sid]])
    Y_train.append(sid2khot[sid])
X_train = np.vstack(X_train)
Y_train = np.vstack(Y_train)
print X_train.shape, Y_train.shape

X_val, Y_val = [],[]
for sid in sid_val:
    X_val.append(dpvecs[sid2rowidx[sid]])
    Y_val.append(sid2khot[sid])
X_val = np.vstack(X_val)
Y_val = np.vstack(Y_val)
print X_val.shape, Y_val.shape

(36917, 500) (36917, 50)
(9229, 500) (9229, 50)


In [40]:
Y_train.sum(axis=0)

array([ 13966.,   8617.,   8183.,   7832.,   6078.,   5923.,   5862.,
         5291.,   4576.,   4551.,   4352.,   4249.,   4209.,   3951.,
         3461.,   3382.,   3285.,   3009.,   2870.,   2784.,   2808.,
         2697.,   2713.,   2534.,   2394.,   2331.,   2312.,   2294.,
         2250.,   2228.,   2189.,   2226.,   2156.,   2098.,   2065.,
         2048.,   1879.,   1837.,   1828.,   1849.,   1842.,   1800.,
         1809.,   1722.,   1667.,   1643.,   1665.,   1604.,   1580.,
        35126.])

In [41]:
Y_val.sum(axis=0)

array([ 3544.,  2119.,  2010.,  1970.,  1556.,  1498.,  1470.,  1341.,
        1170.,  1127.,  1088.,  1071.,  1037.,  1016.,   930.,   849.,
         835.,   780.,   702.,   720.,   695.,   690.,   649.,   650.,
         588.,   576.,   590.,   595.,   564.,   561.,   594.,   549.,
         551.,   545.,   489.,   486.,   497.,   481.,   474.,   450.,
         455.,   447.,   432.,   432.,   410.,   429.,   390.,   405.,
         398.,  8803.])

In [47]:
clfs = []
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
for i in tqdm(range(N_LABELS)):
#     clf = SVC(class_weight='balanced',random_state=1,C=0.2)
    clf =  SGDClassifier(loss='hinge', penalty='l2', 
                   class_weight='balanced',
                   alpha=1e-1, n_iter=200, random_state=1)
    clf.fit(X_train, Y_train[:,i]) 
    clfs.append(clf)

100%|██████████| 50/50 [08:08<00:00,  9.00s/it]


In [48]:
preds = [clfs[i].predict(X_val) for i in xrange(N_LABELS)]    
pred_svm_dp = np.vstack(preds).T
print pred_svm_dp.shape
print map(int, pred_svm_dp.sum(axis=0))
multilabel_evaluate(y_pred=pred_svm_dp,y_true=Y_val)

(9229, 50)
[8053, 5722, 3457, 2928, 2233, 5700, 3102, 2234, 3516, 4000, 3443, 74, 4665, 3628, 2182, 3721, 2271, 237, 3839, 1627, 4038, 2174, 1275, 3080, 9229, 1571, 8994, 1724, 1602, 3144, 8047, 1019, 2989, 0, 2330, 2851, 1415, 0, 6704, 1344, 1092, 7195, 1449, 1606, 3024, 4284, 6485, 9229, 3261, 4220]
precision = 0.148951811811
recall =  0.551990251712
F1 =  0.222779680756
acc =  0.137435773717


In [42]:
clfs = []
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
for i in tqdm(range(N_LABELS)):
#     clf = SVC(class_weight='balanced',random_state=1,C=0.2)
    clf =  SGDClassifier(loss='hinge', penalty='l2', 
                   class_weight='balanced',
                   alpha=1e-2, n_iter=200, random_state=1)
    clf.fit(X_train, Y_train[:,i]) 
    clfs.append(clf)

100%|██████████| 50/50 [05:39<00:00,  6.45s/it]


In [43]:
preds = [clfs[i].predict(X_val) for i in xrange(N_LABELS)]    
pred_svm_dp = np.vstack(preds).T
print pred_svm_dp.shape
print map(int, pred_svm_dp.sum(axis=0))
multilabel_evaluate(y_pred=pred_svm_dp,y_true=Y_val)

(9229, 50)
[8926, 11, 3371, 4719, 0, 3624, 4619, 3316, 4650, 3709, 2215, 0, 0, 5963, 2945, 4799, 5467, 19, 2796, 3432, 4559, 3152, 11, 5863, 71, 9173, 722, 4468, 2933, 4456, 252, 177, 3033, 0, 3536, 7988, 55, 0, 5261, 0, 131, 8269, 1785, 3812, 2904, 3602, 5455, 5955, 2524, 5641]
precision = 0.153250140609
recall =  0.515069144766
F1 =  0.223008806094
acc =  0.13826199174


In [45]:
clfs = []
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
for i in tqdm(range(N_LABELS)):
#     clf = SVC(class_weight='balanced',random_state=1,C=0.2)
    clf =  SGDClassifier(loss='hinge', penalty='l2', 
                   class_weight='balanced',
                   alpha=1e-3, n_iter=200, random_state=1)
    clf.fit(X_train, Y_train[:,i]) 
    clfs.append(clf)
preds = [clfs[i].predict(X_val) for i in xrange(N_LABELS)]    
pred_svm_dp = np.vstack(preds).T
print pred_svm_dp.shape
print map(int, pred_svm_dp.sum(axis=0))
multilabel_evaluate(y_pred=pred_svm_dp,y_true=Y_val)

100%|██████████| 50/50 [06:52<00:00,  7.90s/it]


(9229, 50)
[8856, 8359, 2301, 7038, 61, 22, 7776, 2642, 615, 3654, 2094, 592, 0, 7911, 3833, 1356, 2706, 13, 3701, 0, 4872, 2029, 286, 8248, 5515, 9229, 565, 566, 6877, 5756, 122, 71, 3696, 0, 4335, 5679, 34, 0, 0, 0, 156, 9229, 655, 4576, 2321, 5047, 1978, 8277, 2690, 6678]
precision = 0.151102121967
recall =  0.531224538279
F1 =  0.223866185888
acc =  0.136047328727


### use grid search to find best param for each svm clf

In [49]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
parameters = {
              'alpha': [1, 1e-1, 3e-1, 1e-2, 3e-2, 1e-3, 3e-3, 1e-4],
              'n_iter': [10, 50, 200]}

for i in tqdm(range(N_LABELS)):
    sgd =  SGDClassifier(loss='hinge', penalty='l2', random_state=1, class_weight='balanced')
    clf = GridSearchCV(sgd, parameters, n_jobs=-1)
    clf.fit(X_train, Y_train[:,i]) 
    clfs.append(clf)

preds = [clfs[i].predict(X_val) for i in xrange(N_LABELS)]    
pred_svm = np.vstack(preds).T
print pred_svm.shape
print map(int, pred_svm.sum(axis=0))
multilabel_evaluate(y_pred=pred_svm, y_true=Y_val)

100%|██████████| 50/50 [50:05<00:00, 56.90s/it]


(9229, 50)
[8053, 5722, 3457, 2928, 2233, 5700, 3102, 2234, 3516, 4000, 3443, 74, 4665, 3628, 2182, 3721, 2271, 237, 3839, 1627, 4038, 2174, 1275, 3080, 9229, 1571, 8994, 1724, 1602, 3144, 8047, 1019, 2989, 0, 2330, 2851, 1415, 0, 6704, 1344, 1092, 7195, 1449, 1606, 3024, 4284, 6485, 9229, 3261, 4220]
precision = 0.148951811811
recall =  0.551990251712
F1 =  0.222779680756
acc =  0.137435773717


In [44]:
# clfs = []
# from sklearn.linear_model import SGDClassifier
# from sklearn.svm import SVC
# for i in tqdm(range(N_LABELS)):
#     clf = SVC(class_weight='balanced',random_state=1,C=1.0)
# #     clf =  SGDClassifier(loss='hinge', penalty='l2', 
# # #                    class_weight='balanced',
# #                    alpha=1e-2, n_iter=200, random_state=1)
#     clf.fit(X_train, Y_train[:,i]) 
#     clfs.append(clf)
# preds = [clfs[i].predict(X_val) for i in xrange(N_LABELS)]    
# pred_svm_dp = np.vstack(preds).T
# print pred_svm_dp.shape
# print map(int, pred_svm_dp.sum(axis=0))
# multilabel_evaluate(y_pred=pred_svm_dp,y_true=Y_val)

# SVM + vector generated by our CNN model